# Sensor space analysis: Time domain analysis (ERP/ERF) in MNE-Python

`
Authors:
Britta Westner, Alexandre Gramfort, Denis A. Engemann
`

## Setup

We already know this part from last time:
 

In [ ]:
%matplotlib inline
import os
import numpy as np
import matplotlib as mpl
import matplotlib.pyplot as plt
import mne

mne.set_log_level('error')

## Set the path to the data and load the previously saved epochs

You should have downloaded the `ds000117-practical` folder. We have to let Python know, where to find this folder on your disk. You will have to adjust the path below to reflect your computer and path structure!
You can print the whole path and check the directory to double check it's correct.

In [ ]:
# Change the following path to where the folder ds000117 is on your disk
data_path = os.path.expanduser("~/Documents/teaching/practical_meeg_2022_data/ds000117")

epochs_fname = os.path.join(data_path,
    'derivatives/meg_derivatives/sub-01/ses-meg/meg/sub-01_ses-meg_task-facerecognition_run-01_proc-sss-epo.fif')

In [ ]:
epochs = mne.read_epochs(epochs_fname, proj=False)

In [ ]:
epochs.info

Wait, why are the projections "on"? We did not apply them before saving - can you find out why the seem to be applied now?

## Explore the different conditions in the data

We have seen in the last session, that the data has different events, which mark which stimulus was shown to the participants. The event/trigger structure is as follows:
- 5, 6, 7: famous faces
- 13, 14, 15: unfamiliar faces
- 17, 18, 19: scrambled faces

We have epoched the data according to these events previously. Let's see how we can use these event codes to work with the conditions!

We first look at how we registered the events:

In [ ]:
epochs

We can now query the data based on the event IDs:

In [ ]:
epochs['long']

In [ ]:
epochs['scrambled', 'famous']

In [ ]:
epochs['famous']

We can now for example use this for plotting the epochs based on condition:

In [ ]:
epochs['famous'].plot_image(picks='MEG2143', sigma=1.);

<div class="alert alert-success">
    <b>EXERCISE</b>:
     <ul>
      <li>Can you make the same plot for the other two conditions?</li>
      <li>Is there anything you notice?</li>   
    </ul>
</div>

## Average the epochs to get ERF/ERP 

In [ ]:
evoked = epochs.average()
evoked.apply_proj();

In [ ]:
evoked.info

Let's look at our evoked field and evoked potential!

In [ ]:
evoked.plot();

## Separate the conditions

Let's look at the ERP at channel EEG065 for all three conditions.

In [ ]:
# Remember how we can index the epochs:
for condition in ['famous', 'unfamiliar', 'scrambled']:
    epochs[condition].average().plot(picks='EEG065');

To more easily handle our evoked object, let's make a _list of evoked objects_ for the different conditions we are interested in. Then we can also make it easier for ourselved and compare the ERPs in one plot!

In [ ]:
conditions = ['famous', 'unfamiliar', 'scrambled']
evokeds_list = []
for k, cond in enumerate(conditions):
   evokeds_list.append(epochs[cond].average().apply_proj().crop(-0.5, 1))
   evokeds_list[k].comment = cond  # we want to update the name of the condition for plotting

In [ ]:
mne.viz.plot_compare_evokeds(evokeds_list, picks='EEG065');

But what if we want to see _all_ channels?

In [ ]:
%matplotlib qt
mne.viz.plot_compare_evokeds(evokeds_list, picks='meg', axes='topo');

<div class="alert alert-success">
    <b>EXERCISE</b>:
     <ul>
      <li>Try clicking on the plot, what happens?</li>
      <li>Can you make the same plots for magnetometers and gradiometers?</li>
    </ul>
</div>


## Plot standard deviation across trials

It can be helpful to visualize evoked data with confidence intervals or standard deviations. Let's see how!

For standard deviations across _trials_, there is no ready-to-use MNE-Python function. The reason: Evoked objects do not store the information to compute this metric (i.e., single trials) anymore!

But don't worry: we can plot this ourselves using matplotlib - and gain some knowledge about matplotlib and writing functions on the way!

In [ ]:
# Let's write our own function to compute the standard deviations!
# Since we have to use this operation for each condition, it's handy to write a function for it:

# Everything we need within the function, we have to pass to the function.
# In this case, we need the epochs object, and we want to pick this object for the right condition
# and for channels, so we also pass condition and picks.
def compute_sd(epochs, condition, picks):
    """ Get CI from epochs object for condition."""

    # this gets the data for the condition and channel as an array:
    epochs_data = epochs[condition].get_data(picks=picks)

    # compute the mean first
    epochs_mean = np.mean(epochs_data, axis=0)

    # compute the standard deviation
    epochs_std = np.std(epochs_data, axis=0)

    # add above and below the mean:
    sd_upper = np.squeeze(epochs_mean + epochs_std)
    sd_lower = np.squeeze(epochs_mean - epochs_std)

    # and return what we need
    return sd_lower, sd_upper    


In [ ]:
# we have cropped the evoked, so we also have to crop the epochs!
epochs_crop = epochs.copy().crop(-0.5, 1)

In [ ]:
%matplotlib inline

picks = 'EEG065'  # we plot one channel

# let's first add the lines at zero that MNE-Python has
plt.axhline(y=0., color='black', linewidth=0.75)
plt.axvline(x=0., color='black', linewidth=0.75, linestyle='--')

for ii, condition in enumerate(conditions):

    # first let's compute the standard deviation with our function
    sd_lower, sd_upper = compute_sd(epochs_crop, condition, picks)

    # get the evoked data from the evokeds list
    evoked_data = evokeds_list[ii].get_data(picks=picks)
    times = evokeds_list[ii].times

    # plot the evoked
    plt.plot(times, np.squeeze(evoked_data), linewidth=2, label=condition)

    # plot the standard deviation and add a label once
    plt.fill_between(times, sd_lower, sd_upper, alpha=0.3,
                     label=('standard dev.' if ii==2 else None))



# labels and legend 
plt.ylabel('V')  # note the difference in scaling to MNE-Python!
plt.xlabel('Time (s)')
plt.legend(loc='lower left', framealpha=0.3)

plt.show()


## Plot topographies

Next, we want to plot the topographies at the peak. We want to plot this for all channel types _and_ all conditions. To not wind up with 9 individual plots, let's see how to combine plots into one figure!

In [ ]:
%matplotlib inline
# per channel type, we have different color limits for the topography:
color_lims = [(-150, 150), (0, 45), (-4.5, 4.5)]
channel_types = ['mag', 'grad', 'eeg']
sup_titles = ['Magnetometers', 'Gradiometers', 'EEG']

# we loop across the channel types first:
for ch_type, c_lim, s_title in zip(channel_types, color_lims, sup_titles):

    # Here we build our figure. Note that we ask for 4 subplots! That is because our colorbar
    # needs its own axis.
    fig, axes = plt.subplots(1, 4, figsize=(7, 4), gridspec_kw={'width_ratios': [3, 3, 3, 0.5]})

    # Now we can get the peak time across the conditions for this channel type.
    # We search for the peak time for up to 150 ms after stimulus onset. We will plot 
    # the topography for 100 ms around this peak - see the average parameter in the 
    # plotting function call!
    _, peak_time = evoked.get_peak(ch_type=ch_type, tmax=0.15)

    for ii, (ax, cond) in enumerate(zip(axes, conditions)):
        evokeds_list[ii].plot_topomap(
            ch_type=ch_type, times=peak_time, average=0.1,
            vlim=c_lim, 
            colorbar=(True if ii==2 else False),  # only one colorbar
            axes=(axes[2::] if ii == 2 else ax),  # last plot needs 2 axes
            show=False)

        ax.set_title(cond)

    fig.suptitle('%s, %.3f s' % (s_title, peak_time))

## Write evoked data to disk

In [ ]:
 # note that the file for evoked ends with -ave.fif
evoked_fname = epochs_fname.replace('-epo.fif', '-ave.fif')
evoked_fname

In [ ]:
evoked.save(evoked_fname, overwrite=True) 

You can also write multiple conditions into one file:

In [ ]:
evoked_list_fname = evoked_fname.replace('-ave.fif', '_list-ave.fif')
mne.write_evokeds(evoked_list_fname, evokeds_list, overwrite=True)

### Reading evoked from disk

You can read the data back from the stored `.fif` file:

In [ ]:
evokeds_list = mne.read_evokeds(evoked_fname, baseline=(None, 0), proj=True)

Or give the explicit name of the averaged condition when reading in the list:

In [ ]:
evoked_famous = mne.read_evokeds(evoked_list_fname, condition="famous",
                           baseline=(None, 0), proj=True)
evoked_famous

**Remark:** Did you notice that you can apply some preprocessing on reading the evokeds from disk?

## Contrast the conditions

In [ ]:
# now we can use the condition "face", which consists of famous and unfamiliar
evoked_face = epochs['face'].average().apply_proj()
evoked_scrambled = epochs['scrambled'].average().apply_proj()

# adapt the naming
evoked_face.comment = 'face'
evoked_scrambled.comment = 'scrambled'

In [ ]:
contrast = mne.combine_evoked([evoked_face, evoked_scrambled], [0.5, -0.5])

contrast.comment = 'face - scrambled'

Note that this combines evokeds by taking into account the number of averaged epochs (to scale the noise variance). Let's check out the number of epochs we averaged over:

In [ ]:
print(evoked_face.nave) 
print(evoked_scrambled.nave) 
print(contrast.nave) 

In [ ]:
print(contrast)

Let's plot the contrast together with the ERPs:

In [ ]:
mne.viz.plot_compare_evokeds([evoked_face, evoked_scrambled, contrast], picks='EEG065');

<div class="alert alert-success">
    <b>EXERCISE</b>:
     <ul>
      <li>Can you plot the difference between famous and unfamiliar faces for an occipital MEG sensor?</li>
    </ul>
</div>


## Plot a 3D field map

We can also plot the topography in 3D. Since MNE-Python also plots the head surface here, we need a `-trans.fif` file, that tells us the _transform_ between the MEG device coordinate system and the head surface coordinate system.

In [ ]:
# get the path of the transform file
# usually, this needs to be created by manual coregistration, but we have it all ready!
extra_path = os.path.expanduser("~/Documents/teaching/practical_meeg_2022_data/extra_data_mne/")
trans_fname = os.path.join(extra_path, 'source_recon/sub-01/sub-01-trans.fif')

# we also need the Freesurfer subjects direcory:
subjects_dir = os.path.join(extra_path, 'freesurfer')

Let's double check it exists:

In [ ]:
ls $trans_fname

In [ ]:
# first make the map
maps = mne.make_field_map(evoked_face, trans=trans_fname, ch_type='meg',
                          subject='sub-01', subjects_dir=subjects_dir)
evoked_face.plot_field(maps, time=0.1)